In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
import time

# from mock.tasks import adiciona_carro}
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

session.execute("USE simulacao")

import json
params = json.load(open('./mock/parametros.json'))

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
# import mysql.connector as database
# import os

# connection = database.connect(
#     host="host.docker.internal",
#     port=3306,
#     user="root",
#     password="secret"
# )


# cursor = connection.cursor()
# cursor.execute(query)

In [3]:
number = 1

In [4]:
fps = 30
vel_media = 0
n_vel_media = 0

tempo_medio = 0
n_tempo_medio = 0

def atualiza_media(media_atual, tamanho_atual, media_add, tamanho_add):
    if media_add == None:
        return media_atual
    if tamanho_atual == 0:
        return media_add
    tamanho_total = tamanho_atual + tamanho_add
    return (media_atual/tamanho_total)*tamanho_atual + (media_add/tamanho_total)*tamanho_add

def processa_velocidade_media(batch):
    global vel_media, n_vel_media
    batch = batch.na.fill(0, subset=['velocidade'])
    vel_media_batch = batch.agg(F.mean('velocidade')).collect()[0][0]
    length_batch = batch.select(F.count('velocidade')).collect()[0][0]
    n_vel_media += length_batch
    vel_media = atualiza_media(vel_media, n_vel_media, vel_media_batch, length_batch)
    pass

def processa_tempo_cruzamento(batch):
    global n_tempo_medio, tempo_medio
    batch = batch.filter(col("tempo_em_curso") != 0)
    tempo_medio_batch = batch.agg(F.mean('tempo_em_curso')).collect()[0][0]
    length_batch = batch.select(F.count('tempo_em_curso')).collect()[0][0]
    n_tempo_medio += length_batch
    tempo_medio = atualiza_media(tempo_medio, n_tempo_medio, tempo_medio_batch, length_batch)
    pass

def processa_carro(DadosNovos, DadosCarros, colision_tolerance, colision_tolerance_quad, Parametros):
    time0 = time.time()
    DadosCarros = DadosCarros.drop('aplicaMulta')
    time1 = time.time()
    print('Drop tempo: ',time1-time0)
    time0 = time.time()
    # Renomeia coluna do dado novo
    DadosNovos = DadosNovos.select(F.col('pos_y').alias('posicao_nova'), F.col('rodovia').alias('rodovia_nova'),
                                   F.col('pos_x').alias('faixa_nova'),
                                   F.col('tempo_da_simulacao').alias('tempo_da_simulacao_novo'),F.col('placa'))
    
    data_joined = DadosCarros.join(DadosNovos, on="placa", how='right')

    time1 = time.time()
    print('Join tempo: ',time1-time0)
    time0 = time.time()
    
    CarrosSumidos = DadosCarros.join(DadosNovos, on="placa", how='left_anti')
    processa_tempo_cruzamento(CarrosSumidos)
    data_joined = data_joined.withColumn("tempo_inicio", when(col("tempo_inicio").isNull(), col("tempo_da_simulacao_novo")).otherwise(col("tempo_inicio")))
    data_joined = data_joined.withColumn("tempo_em_curso", col("tempo_da_simulacao_novo") - col("tempo_inicio"))
    data_joined = data_joined.withColumn("rodovia", coalesce(col("rodovia_nova"), col("rodovia")))
    data_joined = data_joined.withColumn("troca_de_faixa", col("faixa_nova") != col("faixa"))
    data_joined = data_joined.withColumn("faixa", coalesce(col("faixa_nova"), col("faixa")))
    data_joined = data_joined.drop("rodovia_nova", "faixa_nova")

    time1 = time.time()
    print('Cruzamento tempo: ',time1-time0)
    time0 = time.time()
    
    data_joined = data_joined.withColumn("diferenca_de_posicao", col("posicao_nova") - col("posicao"))
    
    data_joined = data_joined.withColumn("diferenca_de_horario", (col("tempo_da_simulacao_novo") - col("tempo_da_simulacao"))*fps)

    data_joined = data_joined.withColumnRenamed("velocidade", "velocidade_antiga")

    data_joined = data_joined.withColumn("velocidade", col("diferenca_de_posicao") / col("diferenca_de_horario"))
    processa_velocidade_media(data_joined)
    
    data_joined = data_joined.withColumn("diferenca_de_velocidade", col("velocidade") - col("velocidade_antiga"))

    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))

    data_joined = data_joined.drop("velocidade_antiga", "posicao", "tempo_da_simulacao",
                     "diferenca_de_posicao", "diferenca_de_horario",
                     "diferenca_de_velocidade")

    data_joined = data_joined.withColumnRenamed("posicao_nova", "posicao")

    data_joined = data_joined.withColumnRenamed("tempo_da_simulacao_novo", "tempo_da_simulacao")

    data_joined = data_joined.withColumn("posicao_prevista", col("posicao")\
                           + col("velocidade")*colision_tolerance\
                           + col("aceleracao")*colision_tolerance_quad)

    time1 = time.time()
    print('With tempo: ',time1-time0)
    time0 = time.time()

    Velocidades_Maximas = Parametros.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
    Aceleracoes_Maximas = Parametros.select(F.col('rodovia'), 0.8*F.col("AceleracaoMaxima"))
    Aceleracoes_Maximas = Aceleracoes_Maximas.withColumnRenamed("(AceleracaoMaxima * 0.8)", "AceleracaoMaxima")
    
    data_joined = data_joined.join(Velocidades_Maximas, on="rodovia", how="left")
    data_joined = data_joined.join(Aceleracoes_Maximas, on="rodovia", how="left")

    acima_vel_df = data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_antigo'))
    data_joined = data_joined.withColumn("acima_vel", F.when(F.abs(data_joined["velocidade"]) > F.abs(fps/data_joined["VelocidadeMaxima"]), 1).otherwise(0))
    data_joined = data_joined.withColumn("acima_acel", F.when(F.abs(data_joined["aceleracao"]) > F.abs(fps/data_joined["AceleracaoMaxima"]), 1).otherwise(0))

    acima_vel_df = acima_vel_df.join(data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_novo')), on='placa', how="left")
    acima_vel_df = acima_vel_df.withColumn("aplicaMulta", (F.col('acima_vel_antigo')==0) &  (F.col('acima_vel_novo')==1) )



    data_joined = data_joined.join(acima_vel_df.select(F.col('placa'), F.col('aplicaMulta')), on='placa', how='left')

    data_joined = data_joined.drop("VelocidadeMaxima")
    data_joined = data_joined.drop("AceleracaoMaxima")

    time1 = time.time()
    print('Acima tempo: ',time1-time0)
    time0 = time.time()

    window_spec = Window.partitionBy("rodovia", "faixa").orderBy('posicao')
    
    # Use lag function with the window specification
    lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec)
    lead_column = lead(col("posicao_prevista")).over(window_spec) - col("posicao_prevista")

    
    # Add the lag column to the DataFrame
    data_joined = data_joined.withColumn("Risco_Colisão", when(((lag_column < 0) & (col("rodovia") == lag(col("rodovia")).over(window_spec)) & (col("faixa") == lag(col("faixa")).over(window_spec)))| ((lead_column < 0) & (col("rodovia") == lead(col("rodovia")).over(window_spec)) & (col("faixa") == lead(col("faixa")).over(window_spec))), 1).otherwise(0))

    time1 = time.time()
    print('Window tempo: ',time1-time0)
    time0 = time.time()
    
    # Show the result
    return data_joined
    

In [5]:
df_multas = ss.createDataFrame([], "placa: string, tempo_da_simulacao: int, multa_numero : int")
def aplica_multa(df):
    global df_multas
    df = df.filter(F.col('aplicaMulta') == True)
    df = df.select(F.col('placa'),F.col('tempo_da_simulacao'))
    df2 = df_multas.groupBy('placa').agg(F.count('placa').alias('multa_numero'))
    df2 = df2.withColumn('multa_numero',F.col('multa_numero')+1)    
    df2 = df2.join(df,['placa'],how='right')
    df2 = df2.withColumn('multa_numero', F.when(F.isnull('multa_numero'), 1).otherwise(F.col('multa_numero')))
    df_multas = df2.union(df_multas)

In [11]:
df_perigosa = ss.createDataFrame([], "placa: string, tempo_da_simulacao: int, perigosa_numero : int, tipo: string")
# def perigosa_acel(df,df_perigosa):
#     df = df.filter(F.col('acima_acel') == 1).select(F.col('placa'),F.col('tempo_da_simulacao'))
#     window = Window.partitionBy("placa")
#     df2 = df_multas.withColumn("perigosa_numero", count("placa").over(window) + 1)
#     df2 = df2.join(df,['placa'],how='right')
#     df_perigosa = df2.withColumn('perigosa_numero', F.when(F.isnull('perigosa_numero'), 1).otherwise(F.col('perigosa_numero'))).withColumn('tipo',F.lit('acima_acel')).union(df_perigosa) 

# def perigosa_faixa(df,df_perigosa):
#     df = df.filter(F.col('troca_de_faixa') == True).select(F.col('placa'),F.col('tempo_da_simulacao'))
#     window = Window.partitionBy("placa")
#     df2 = df_multas.withColumn("perigosa_numero", count("placa").over(window) + 1)
#     df2 = df2.join(df,['placa'],how='right')
#     df_perigosa = df2.withColumn('perigosa_numero', F.when(F.isnull('perigosa_numero'), 1).otherwise(F.col('perigosa_numero'))).withColumn('tipo',F.lit('acima_acel')).union(df_perigosa) 

def perigosa(df,col):
    global df_perigosa
    df = df.filter(F.col(col) == 1).select(F.col('placa'),F.col('tempo_da_simulacao'))
    window = Window.partitionBy("placa")
    df2 = df_multas.withColumn("perigosa_numero", count("placa").over(window) + 1)
    df2 = df2.join(df,['placa'],how='right')
    df_perigosa = df2.withColumn('perigosa_numero', F.when(F.isnull('perigosa_numero'), 1).otherwise(F.col('perigosa_numero'))).withColumn('tipo',F.lit(col)).union(df_perigosa) 

In [7]:
p = [[key]+list(params[key].values()) for key in params.keys()]
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))
df_old = ss.createDataFrame([], "placa: string, posicao: int, faixa: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean, aplicaMulta: boolean, tempo_em_curso: int,tempo_inicio: int")

def pipeline(first=1,last=session.execute("select count(*) from simulacao").one()[0]):
    global p
    global df_old
    number = first
    while number < last:
        print(number)
        time0 = time.time()
        a, b = interval
        query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao BETWEEN {a} AND {b} ALLOW FILTERING;"
        r = list(session.execute(query))
        time1 = time.time()
        print('Leitura do banco tempo: ',time1-time0)
        time0 = time.time()
        if r != []:
            df = ss.createDataFrame(r)
            time1 = time.time()
            print('Criação do df tempo: ',time1-time0)
            time0 = time.time()
            df_old = processa_carro(df, df_old, 1, 0.5, p)
            time1 = time.time()
            print('Processa carros tempo: ',time1-time0)
            time0 = time.time()
            aplica_multa(df_old)
            time1 = time.time()
            print('Aplica multa tempo: ',time1-time0)
            time0 = time.time()
            perigosa(df_old,'acima_vel')
            perigosa(df_old,'acima_acel')
            perigosa(df_old,'troca_de_faixa')
            time1 = time.time()
            print('Perigosa tempo: ',time1-time0)
            time0 = time.time()
        
        number+=1
        # last = session.execute("select count(*) from simulacao").one()[0]

pipeline(168,171)

168
Leitura do banco tempo:  0.008158683776855469
Criação do df tempo:  0.042145729064941406
Drop tempo:  0.059146881103515625
Join tempo:  0.1045229434967041
Cruzamento tempo:  4.07969069480896
With tempo:  2.0627195835113525
Acima tempo:  0.3397691249847412
Window tempo:  0.09449648857116699
Processa carros tempo:  6.740910291671753
Aplica multa tempo:  0.13347411155700684
Perigosa tempo:  1.0925843715667725
169
Leitura do banco tempo:  0.007907867431640625
Criação do df tempo:  0.03211212158203125
Drop tempo:  0.008744001388549805
Join tempo:  0.027940988540649414
Cruzamento tempo:  6.229593992233276
With tempo:  3.8141367435455322
Acima tempo:  0.43222951889038086
Window tempo:  0.06929802894592285
Processa carros tempo:  10.58243727684021
Aplica multa tempo:  0.29760122299194336
Perigosa tempo:  2.222799301147461
170
Leitura do banco tempo:  0.006751060485839844
Criação do df tempo:  0.02100539207458496
Drop tempo:  0.015866518020629883
Join tempo:  0.05795168876647949
Cruzamento 

In [8]:
df_old.toPandas()

,placa,rodovia,faixa,aceleracao,posicao_prevista,acima_vel,tempo_em_curso,tempo_inicio,troca_de_faixa,acima_acel,Risco_Colisão,posicao,tempo_da_simulacao,velocidade,aplicaMulta
0,URU6M78,BR-116,475.0,NaN,NaN,0,0.0,170.0,None,0,0,106.50,170.0,NaN,False
1,URU2Y34,BR-116,655.0,-0.002222,-31.834444,0,2.0,168.0,False,0,0,-31.50,170.0,-0.333333,False
2,BRA2A34,BR-116,745.0,NaN,NaN,1,1.0,169.0,True,0,0,842.75,170.0,21.006667,True
3,PAR5Z67,BR-116,835.0,NaN,NaN,0,1.0,169.0,False,0,0,622.50,170.0,-2.200000,False


In [9]:
df_perigosa.toPandas()

,placa,perigosa_numero,tempo_da_simulacao,tipo
0,BRA2A34,2,170.0,troca_de_faixa
1,BRA2A34,2,170.0,acima_vel


In [10]:
df_multas.show()

+-------+------------+------------------+
|  placa|multa_numero|tempo_da_simulacao|
+-------+------------+------------------+
|BRA2A34|           1|             170.0|
+-------+------------+------------------+



In [254]:
a, b = 20, 50
query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao >= {a} AND tempo_da_simulacao <= {b} ALLOW FILTERING;"
r = list(session.execute(query))
if r != []:
    df = ss.createDataFrame(r)

In [255]:
df.count()

118

In [256]:
df.show()

+-------+-------+------------------+-----+------------------+
|  placa|rodovia|tempo_da_simulacao|pos_x|             pos_y|
+-------+-------+------------------+-----+------------------+
|BRA2A34| BR-116|              38.0|745.0|             835.4|
|BRA2A34| BR-116|              41.0|745.0|             795.5|
|BRA2A34| BR-116|              43.0|745.0|            798.55|
|BRA2A34| BR-116|              49.0|745.0|            854.15|
|GUY5L67| BR-116|              21.0|565.0|             76.85|
|GUY5L67| BR-116|              23.0|565.0|             108.0|
|GUY5L67| BR-116|              25.0|565.0|             65.75|
|GUY5L67| BR-116|              27.0|565.0|103.10000000000001|
|GUY5L67| BR-116|              28.0|565.0|185.35000000000005|
|GUY5L67| BR-116|              29.0|565.0|              73.0|
|GUY5L67| BR-116|              34.0|565.0|             98.35|
|GUY5L67| BR-116|              38.0|565.0|229.60000000000008|
|GUY5L67| BR-116|              39.0|565.0|              89.3|
|GUY5L67

In [257]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, col

Velocidades_Maximas = p.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
Aceleracoes_Maximas = p.select(F.col('rodovia'), 0.8*F.col("AceleracaoMaxima"))
Aceleracoes_Maximas = Aceleracoes_Maximas.withColumnRenamed("(AceleracaoMaxima * 0.8)", "AceleracaoMaxima")

collision_tolerance = 1
collision_tolerance_quad = 0.5
windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")

df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
df = df.withColumn("prev_tempo_da_simulacao", lag("tempo_da_simulacao", 1).over(windowSpec))
df = df.withColumn("vel_y", (col("pos_y") - col("prev_pos_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
df = df.withColumn("prev_vel_y", lag("vel_y", 1).over(windowSpec))
df = df.withColumn("acel_y", (col("vel_y") - col("prev_vel_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))

df = df.withColumn("posicao_prevista", col("pos_y") + col("vel_y") * (collision_tolerance) + col("acel_y") * collision_tolerance_quad)

window_spec_rf = Window.partitionBy("rodovia", "pos_x").orderBy('pos_y')
lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec_rf)
lead_column = lead(col("posicao_prevista")).over(window_spec_rf) - col("posicao_prevista")

# Add the lag column to the DataFrame
df = df.withColumn("Risco_Colisão", when(((lag_column < 0) & (col("rodovia") == lag(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lag(col("pos_x")).over(window_spec_rf)))| ((lead_column < 0) & (col("rodovia") == lead(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lead(col("pos_x")).over(window_spec_rf))), 1).otherwise(0))

df = df.join(Velocidades_Maximas,on='rodovia',how='left')
df = df.join(Aceleracoes_Maximas,on='rodovia',how='left')

df = df.withColumn('acima_vel',F.abs(col('vel_y'))>F.abs(col('VelocidadeMaxima')))
df = df.withColumn('acima_acel',F.abs(col('acel_y'))>F.abs(col('AceleracaoMaxima')))

df = df.withColumn("troca_faixa", col("pos_x") != lag("pos_x", 1).over(windowSpec))

# contador de trocas

df = df.withColumn('multado',((F.col('acima_vel') == 1) & (lag('acima_vel').over(windowSpec) == 0)))

windowSpec = Window.partitionBy("placa",'rodovia').orderBy('tempo_da_simulacao')

df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
df = df.withColumn("on_road", (((col("pos_y") > 0) & (col('pos_y') < 800)) & (col("prev_pos_y") > 0)).cast("int"))
df = df.withColumn('time_on_road',F.lit(0))
# df = df.withColumn("time_on_road", col("on_road") * (1 + lag("time_on_road", 1).over(windowSpec)))

# windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao").rowsBetween(Window.unboundedPreceding, 0)

# # df = df.withColumn("cumulative_sum", F.sum(col("on_road")).over(windowSpec))

# # # windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")

# df = df.withColumn("reset", when(col("time_on_road") != lag("time_on_road", 1).over(windowSpec), 1).otherwise(0))
# df = df.withColumn("group", F.sum("reset").over(windowSpec))
# df = df.withColumn("cumulative_sum", F.sum(col("placa")).over(windowSpec.partitionBy("group")))

In [258]:
dfp = df.toPandas()
dfp[dfp['placa'] == 'PAR5Z67'].sort_values(by=['tempo_da_simulacao'], ascending=True)


,rodovia,placa,tempo_da_simulacao,pos_x,pos_y,prev_pos_y,prev_tempo_da_simulacao,vel_y,prev_vel_y,acel_y,posicao_prevista,Risco_Colisão,VelocidadeMaxima,AceleracaoMaxima,acima_vel,acima_acel,troca_faixa,multado,on_road,time_on_road
52,BR-116,PAR5Z67,20.0,655.0,716.50,NaN,NaN,NaN,NaN,NaN,NaN,0,9,0.12,None,None,None,None,NaN,0
53,BR-116,PAR5Z67,21.0,655.0,701.00,716.50,20.0,-15.500,NaN,NaN,NaN,0,9,0.12,True,None,False,None,1.0,0
54,BR-116,PAR5Z67,22.0,655.0,786.80,701.00,21.0,85.800,-15.500,101.3000,923.25000,1,9,0.12,True,True,False,False,1.0,0
55,BR-116,PAR5Z67,23.0,655.0,803.90,786.80,22.0,17.100,85.800,-68.7000,786.65000,1,9,0.12,True,True,False,False,0.0,0
56,BR-116,PAR5Z67,24.0,655.0,698.05,803.90,23.0,-105.850,17.100,-122.9500,530.72500,1,9,0.12,True,True,False,False,1.0,0
57,BR-116,PAR5Z67,25.0,655.0,662.00,698.05,24.0,-36.050,-105.850,69.8000,660.85000,1,9,0.12,True,True,False,False,1.0,0
58,BR-116,PAR5Z67,30.0,655.0,762.65,662.00,25.0,20.130,-36.050,11.2360,788.39800,1,9,0.12,True,True,False,False,1.0,0
59,BR-116,PAR5Z67,31.0,655.0,766.55,762.65,30.0,3.900,20.130,-16.2300,762.33500,1,9,0.12,False,True,False,False,1.0,0
60,BR-116,PAR5Z67,32.0,655.0,782.65,766.55,31.0,16.100,3.900,12.2000,804.85000,0,9,0.12,True,True,False,True,1.0,0
61,BR-116,PAR5Z67,33.0,655.0,629.50,782.65,32.0,-153.150,16.100,-169.2500,391.72500,1,9,0.12,True,True,False,False,1.0,0
